In [1]:
import numpy as np
import os
import re
import matplotlib.colors as colors

from scipy.ndimage import filters

In [2]:
i = 0
l = np.load('l{0}.npy'.format(i)).astype(np.float)
gt = np.load('g{0}.npy'.format(i)).astype(np.float)
costs = np.load('b{0}.npy'.format(i)).astype(np.float)
box_costs = (filters.uniform_filter(costs,[7,7,1],mode='nearest')*(7*7)).astype(np.int)

In [126]:
def subpixel_f(left,cent,right):
    num = right - left
    den = (cent-left) if (right < left) else (cent-right)
    return 0.0 if den == 0 else 0.5*(num/float(den))
def second_peak_f(corr_v,best,max_cost):
    sp = max_cost
    for i in xrange(0,corr_v.shape[0]-1):
        if i!=best and corr_v[i] < corr_v[i-1] and corr_v[i] < corr_v[i+1]:
            if corr_v[i] < sp:
                sp = corr_v[i]
    return sp
                
        
def stereo_features(costs,gt):
    height,width,mdisp = costs.shape
    
    best = np.argmin(costs,2)
    bestr = np.zeros_like(best)
    bestlS = np.zeros_like(best).astype(np.float)
    bestrS = np.zeros_like(best).astype(np.float)
    l_r_match = np.zeros_like(best).astype(np.float)

    for ri,row in enumerate(costs):
        for ci,col in enumerate(row):
            lim = min(width,ci+mdisp)-ci
            rv, r_min = min([(costs[ri,ci+i,i],i) for i in xrange(0,lim)])
            bestr[ri,ci] = r_min
            shift = 0.0
            if r_min >0 and r_min < lim-1:
                rminl = r_min -1
                rminr = r_min + 1
                shift = subpixel_f(costs[ri,ci+rminl,rminl],costs[ri,ci+r_min,r_min],costs[ri,ci+rminr,rminr])
            bestrS[ri,ci] = r_min + shift
    for ri,row in enumerate(costs):
        for ci,col in enumerate(row):
            l_min = best[ri,ci]
            shift = 0.0
            if l_min >0 and l_min < mdisp-1:
                lminl = l_min -1
                lminr = l_min + 1
                shift = subpixel_f(costs[ri,ci,lminl],costs[ri,ci,l_min],costs[ri,ci,lminr])
            bestlS[ri,ci] = l_min + shift
    for ri,row in enumerate(costs):
        for ci,col in enumerate(row):
            l_min = best[ri,ci]
            l_minS = bestlS[ri,ci]
            r_minS = bestrS[ri,ci-l_min]
            l_r_match[ri,ci] = abs(l_minS-r_minS)
    
    costs = costs.reshape([-1,mdisp])
    gt = gt.reshape([-1])
    l_r_match = l_r_match.reshape([-1])
    n = gt.shape[0]
    
    idx = np.where(gt != -2)[0]
    costs = costs[gt != -2]
    max_cost = costs.max()
    gt = gt[gt != -2]
    l_r_match = l_r_match[gt != -2]
    best = np.argmax(costs,1)
    
    valid = (gt != -1).astype(np.int)
    est_correct = (best == np.round(gt)).astype(np.int)
    
    minv = np.min(costs,1)
    maxv = np.max(costs,1)
    median = np.median(costs,1)
    left_d  = np.array([costs[idx,v-1]-costs[idx,v] if v > 0       else max_cost for idx,v in enumerate(best)])
    right_d = np.array([costs[idx,v+1]-costs[idx,v] if v < mdisp-1 else max_cost for idx,v in enumerate(best)])
    second_peak  = np.array([second_peak_f(costs[idx,:],v,max_cost) for idx,v in enumerate(best)])
    
    return np.vstack([valid,est_correct,minv,maxv,median,left_d,right_d,l_r_match,second_peak])
    
ft = stereo_features(box_costs,gt).T

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:59: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 356128 but corresponding boolean dimension is 354918


In [131]:
ft.shape
ft[0,:]
x_min = np.min(ft,0)
x_max = np.max(ft,0)
x_mean = np.mean(ft,0)
print x_min
print x_max
print x_mean

#x_initial = np.array([0,512,256,50,50,0.

[   0.    0.    0.    0.    0. -553. -511.    0.    0.]
[  1.00000000e+00   1.00000000e+00   7.22000000e+02   1.13200000e+03
   9.01000000e+02   1.13200000e+03   1.13200000e+03   7.20000000e+01
   1.13200000e+03]
[  9.16025674e-01   5.91685967e-04   2.42227438e+02   7.32355214e+02
   5.41955984e+02   3.44312771e+00  -1.45779786e+01   4.05303926e+00
   2.49582163e+02]


In [ ]:
data = []
i = 0
     
while True:
    try:
        l = np.load('l{0}.npy'.format(i)).astype(np.float)
        gt = np.load('g{0}.npy'.format(i)).astype(np.float)
        costs = np.load('b{0}.npy'.format(i)).astype(np.float)
        #acosts = np.load('a{0}.npy'.format(i)).astype(np.float)
        box_costs = (filters.uniform_filter(costs,[7,7,1],mode='nearest')*(7*7)).astype(np.int)

        best = np.argmin(box_costs,2)
    except:
        break